In [2]:
import pandas as pd

In [3]:
import math

def harmonize_agrupacion_id(agrupacion_id):
    if pd.isna(agrupacion_id):
        return "000000"
    else:
        try:
            # Attempt to convert string to float and then to integer
            return str(int(float(agrupacion_id))).zfill(6)
        except ValueError:
            # If conversion fails, return original string value
            return agrupacion_id

In [4]:
eleccion = pd.read_csv('./../datos/BD/eleccion_table.csv')
eleccion

,eleccion_id,año,eleccion_tipo,recuento_tipo,padron_tipo
0,1,2011,PASO,PROVISORIO,NORMAL
1,10,2017,PASO,PROVISORIO,NORMAL
2,9,2017,PASO,PROVISORIO,COMANDO
3,8,2017,GENERAL,PROVISORIO,NORMAL
4,7,2017,GENERAL,PROVISORIO,COMANDO
5,5,2015,GENERAL,PROVISORIO,NORMAL
6,4,2015,BALLOTAGE,PROVISORIO,NORMAL
7,0,2011,GENERAL,PROVISORIO,NORMAL
8,6,2015,PASO,PROVISORIO,NORMAL
9,3,2013,PASO,PROVISORIO,NORMAL


In [5]:
cargo = pd.read_csv('./../datos/BD/cargo_table.csv')

agrup_lista = pd.read_csv('./../datos/BD/agrupacion_lista_table.csv')
agrup_lista['agrupacion_id'] = agrup_lista['agrupacion_id'].apply(harmonize_agrupacion_id)
agrup_lista

agrup = agrup_lista[['eleccion_id',	'agrupacion_id']].drop_duplicates()
agrup

,eleccion_id,agrupacion_id
0,1,000509
4,1,000003
8,1,000037
11,1,000133
12,1,000057
...,...,...
2951,12,02-737
2952,12,02-1912
2953,12,02-884
2954,12,02-888


agrupacion_id
000000                                                [nan]
000001    [MOVIMIENTO DE INTEGRACION Y DESARROLLO, Movim...
000002    [JUSTICIALISTA, Partido Justicialista, PARTIDO...
000003    [UNION CIVICA RADICAL, Unión Cívica Radical, U...
000005    [DEMOCRATA CRISTIANO, Partido Demócrata Cristi...
                                ...                        
24-187                          [PARTIDO SOCIAL PATAGONICO]
24-203                                     [PARTIDO OBRERO]
24-501                                    [FRENTE DE TODOS]
24-502    [JUNTOS POR EL CAMBIO, JUNTOS POR EL CAMBIO TI...
24-503                          [VAMOS TODOS A VIVIR MEJOR]
Name: agrupacion_nombre, Length: 644, dtype: object

In [6]:
import glob
import os

## list of csv files
csv_files = glob.glob('./../datos/BD/votos_eleccion_*_table.csv')

## list of dataframes
dfs = [pd.read_csv(f) for f in csv_files]
data = pd.concat(dfs, ignore_index=True)

/tmp/ipykernel_179838/2625637491.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_179838/2625637491.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_179838/2625637491.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_179838/2625637491.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_179838/2625637491.py:8: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_179838/2625637491.py:8: DtypeWarning: Columns (3) have mixed types. 

In [7]:
data['agrupacion_id'] = data['agrupacion_id'].apply(harmonize_agrupacion_id)


In [8]:
table = data.groupby(['eleccion_id', 'cargo_id', 'agrupacion_id']).agg({'votos_cantidad': 'sum'}).reset_index()
table.head()

,eleccion_id,cargo_id,agrupacion_id,votos_cantidad
0,0,1,000000,908675
1,0,1,000047,396171
2,0,1,000131,11593023
3,0,1,000132,1264609
4,0,1,000133,1714385


In [18]:
table2 = table.groupby(['eleccion_id', 'cargo_id'])['votos_cantidad'].sum().reset_index()
table2 = table2.merge(eleccion).merge(cargo)   
table2 = table2[['año', 'eleccion_tipo', 'cargo_nombre', 'votos_cantidad']]
table2 = table2.sort_values('votos_cantidad', ascending = False)
table2.style.bar(subset=['votos_cantidad'], align='mid', color='#d65f5f', vmax=30000000)

,año,eleccion_tipo,cargo_nombre,votos_cantidad
6,2019,GENERAL,Presidente,26582166
35,2019,GENERAL,Diputado Nacional,26430440
2,2015,BALLOTAGE,Presidente,25738560
31,2017,GENERAL,Diputado Nacional,25394746
3,2015,GENERAL,Presidente,25223502
94,2015,GENERAL,Parlamentarios Mercosur Nacionales,25054196
28,2015,GENERAL,Diputado Nacional,24961718
96,2015,GENERAL,Parlamentarios Mercosur Provinciales,24899831
8,2019,PASO,Presidente,24707587
26,2013,GENERAL,Diputado Nacional,24464393


In [ ]:
## n largest by eleccion_id and cargo_id

table2 = table.groupby(['eleccion_id', 'cargo_id']).apply(lambda x: x.nlargest(3, 'votos_cantidad')).reset_index(drop=True)
table2 = table2.merge(eleccion).merge(agrup).merge(cargo)
table2 = table2[['año', 'eleccion_tipo', 'cargo_nombre', 'agrupacion_id', 'votos_cantidad']]
table2 = table2.sort_values('votos_cantidad', ascending = False)

table2['cumsum'] = 100*table2.votos_cantidad.cumsum()/table2.votos_cantidad.sum()

import numpy as np

# style the dataframe with a bar chart for the 'votos_cantidad' column
styled_table2 = (table2.style
                .bar(subset=['votos_cantidad'], align='mid', color='#d65f5f', vmax=15000000)
                .set_caption('Resultados elecciones')
                )

# display the styled dataframe
styled_table2


,año,eleccion_tipo,cargo_nombre,agrupacion_id,votos_cantidad,cumsum
6,2015,BALLOTAGE,Presidente,000135,12903301,2.347879
16,2019,GENERAL,Presidente,000136,12473709,4.617589
7,2015,BALLOTAGE,Presidente,000131,12198441,6.837211
20,2019,PASO,Presidente,000136,11622428,8.952023
0,2011,GENERAL,Presidente,000131,11593023,11.061484
17,2019,GENERAL,Presidente,000135,10470607,12.966710
9,2015,GENERAL,Presidente,000131,9002242,14.604754
228,2015,GENERAL,Parlamentarios Mercosur Nacionales,000131,8590240,16.167830
12,2015,PASO,Presidente,000131,8424749,17.700793
10,2015,GENERAL,Presidente,000135,8382610,19.226089


In [ ]:
agrup

,eleccion_id,agrupacion_id
0,1,000509
4,1,000003
8,1,000037
11,1,000133
12,1,000057
...,...,...
2951,12,02-737
2952,12,02-1912
2953,12,02-884
2954,12,02-888


In [ ]:
table2.merge(eleccion).head()

,eleccion_id,cargo_id,agrupacion_id,votos_cantidad,año,eleccion_tipo,recuento_tipo,padron_tipo
0,0,1,000131,11593023,2011,GENERAL,PROVISORIO,NORMAL
1,0,1,000134,3624518,2011,GENERAL,PROVISORIO,NORMAL
2,0,1,000137,2395056,2011,GENERAL,PROVISORIO,NORMAL
3,0,1,000133,1714385,2011,GENERAL,PROVISORIO,NORMAL
4,0,1,000132,1264609,2011,GENERAL,PROVISORIO,NORMAL


In [ ]:
pd.options.display.max_rows = 999

In [ ]:
table2.merge(eleccion).merge(agrup)

,eleccion_id,cargo_id,agrupacion_id,votos_cantidad,año,eleccion_tipo,recuento_tipo,padron_tipo,agrupacion_nombre
0,0,1,000131,11593023,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
1,0,2,000131,5001099,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
2,0,3,000131,7966496,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
3,0,4,000131,4604459,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
4,0,5,000131,2343958,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
5,0,6,000131,2264602,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
6,0,10,000131,2922979,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente para la Victoria
7,0,1,000134,3624518,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente Amplio Progresista
8,0,2,000134,1078328,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente Amplio Progresista
9,0,3,000134,2177191,2011,GENERAL,PROVISORIO,NORMAL,Alianza Frente Amplio Progresista
